In [0]:
%sql
SELECT * FROM `workspace`.`default`.`superstore`;

In [0]:
# from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Load the dataset from Databricks table
df = spark.read.table("workspace.default.superstore")

In [0]:
# Initial Data Inspection

df.printSchema()

In [0]:
df.show(5,truncate=False)

In [0]:
print(f"Total Rows: {df.count()}")

In [0]:
columns = df.columns

print(columns)

In [0]:
#Duplicate Check
df.groupBy(df.columns).count().filter(F.col('count') > 1).count()

In [0]:
#Null Check
df.filter(F.col("Sales").isNull()).count()

In [0]:
# NaN Check
df.filter(F.isnan("Sales")).count()

In [0]:
df.select("Ship Mode").distinct().show()

Conditional Functions

In [0]:
# Coalesce

# Define a dictionary mapping the column name to its desired default value
default_values = {
    "Order ID": "MISSING_ID",
    "Customer ID": "UNKNOWN_CUST",
    "Ship Mode" : "UNKNOWN Shipping Mode",
    "Customer Name" : "Anonymous Buyer",
    "Segment" : "UNKNOWN Segment",
    "Country" : "UNKNOWN Country",
    "City" : "UNKNOWN City",
    "State" : "UNKNOWN State",
    "Postal Code" : 0000,
    "Region" : "UNKNOWN Region",
    "Product ID" : "UNKNOWN Product",
    "Category" : "UNKNOWN Category",
    "Sub-Category" : "UNKNOWN Sub-Category",
    "Product Name" : "UNKNOWN Product Name",
    "Sales" : 0.0,
    "Quantity" : 0,
    "Discount" : 0.0,
    "Profit" : 0.0
}

df_cleaned = df # Start with the original DataFrame

for col_name, default_val in default_values.items():
    # F.lit() is crucial here to treat the default_val as a constant column
    df_cleaned = df_cleaned.withColumn(
        col_name, 
        F.coalesce(F.col(col_name), F.lit(default_val))
    )

df_cleaned.select("Discount", "Ship Mode", "Customer Name").show(10)

In [0]:
# When

df_cleaned = df_cleaned.withColumn(
    "HighSalesFlag",
    F.when(F.col("Sales") > 500, "High")
     .otherwise("Low")
)

df_cleaned.select("HighSalesFlag").show(5)

In [0]:
df_cleaned = df_cleaned.withColumn(
    "ProfitCategory",
    F.when(F.col("Profit") > 100, "High Profit")
     .when((F.col("Profit") > 0) & (F.col("Profit") <= 100), "Low Profit")
     .otherwise("Loss")
)

df_cleaned.select("Profit", "ProfitCategory").show(5)

In [0]:
df_cleaned = df_cleaned.withColumn(
    "OrderLabel",
    F.when(F.col("Sales") > 1000, "Big Order")
     .when((F.col("Quantity") > 5) & (F.col("Discount") > 0), "Bulk Discount Order")
     .otherwise("Normal Order")
)

df_cleaned.select("OrderLabel").show(5)